In [1]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from bootstrapping.bootstrapping import BootStrapping
from make_representations.representation_maker import Metagenomic16SRepresentation
from classifier.classical_classifiers import RFClassifier,SVM
from classifier.DNN import DNNMutliclass16S
import pandas as pd

Using TensorFlow backend.
/home/nuria/.local/share/virtualenvs/MicroPheno-f1MtFhBm/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-f1MtFhBm/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-f1MtFhBm/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

# Crohn's Disease

### Ler fastaq

In [2]:
# LER OS FASTAQ

# fasta_files: arquivos fastaq
# mapping: orde na que foron lidos/iterados
#fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq')
fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq', 
                only_files=["ERR1368879.fastq", "ERR1368880.fastq", "ERR1368881.fastq", 
                            "ERR1368882.fastq", "ERR1368883.fastq", "ERR1368884.fastq",
                            "ERR1368885.fastq", "ERR1368886.fastq", "ERR1368887.fastq",
                            "ERR1368888.fastq", "ERR1368889.fastq", "ERR1368890.fastq"])

### 1. Bootstrapping

In [9]:
# CONSTRUIR OBXECTO PARA O BOOTSTRAPPING
'''
input_dir: input directory or a list of files
output_dir: a directory to generate the output files
sampling_sizes: a list of sampling sizes (N's)
n_resamples: number of resamples from each file (N_R = 10)
seqtype: file suffixes fastq or fasta etc.
M: number of files from the directory to make samplings from (M)
'''

BS=BootStrapping(fasta_files,'../../crohns_disease/results/bootstrapping/', seqtype='fastq',
                 sampling_sizes=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], n_resamples=10, M=10)

In [ ]:
# add k-mer samplings
for k in [3,4,5,6,7,8]:
    print(k,'-mer bootstrapping..')
    BS.add_kmer_sampling(k)

In [19]:
# GARDAR RESULTADOS EN ARQUIVO 'bootstrapping_crohns_disease_prediction' NA RUTA QUE SE LLE INDICOU AO DEFINIR A INSTANCIA BS
BS.save_me('bootstrapping_crohns_disease_prediction')

### 2. k-mer representations

In [ ]:
# ENTRENAMENTO COS MELLORES TAMAÑOS DE K-MER (K) E OS SEUS CORRESPONDENTES TAMAÑOS DE MOSTRA (N) 
# (SEGÚN OS RESULTADOS DE BOOTSTRAPPING)

# dictionary of k-mers and their sampling values
# 6:[100,1000,2000,5000,10000,-1]  :  k-mer:[sampling values]      (-1 means using all sequences)
#sampling_dict={3:[20],4:[100],5:[500],6:[100,1000,2000,5000,10000,-1],7:[5000],8:[8000]}
sampling_dict={6:[5000]}

# number of cores
nump_p=20

# Para cada arquivo fastaq, creamos unha representación por cada posible combinación de tamaño de k-mer e tamaño de mostra, e gardamos os resultados
for k in sampling_dict.keys():  # for each k-mer
    for N in sampling_dict[k]:  # select a sampling size N
        print(k,'-mers with sampling size ',N)
        
        RS=Metagenomic16SRepresentation(fasta_files, mapping, N, nump_p)
        RS.generate_kmers_all(k, save='../../crohns_disease/datasets/PRUEBA-dataset_'+str(k)+'-mers_rate_complete1359_seq_'+str(N))

### 3. Classifers

#### Classical Classifiers

In [2]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

# X = matriz con datos:  (nº mostra, nº k-mers da secuencia) Abundancia relativa do k-mer na secuencia
#     en total temos 1359 mostras e 4096 6-mers
# Y = lista de resultados reais:  (CD , not CD)
#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz')
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz')
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')


# vemos os datos de X
#dataset=pd.read_csv("../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.csv")
#dataset

In [3]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Random Forest ################
MRF = RFClassifier(X, Y)
MRF.tune_and_eval('../../crohns_disease/results-PRUEBA/RFclassifier', n_fold=2)

[['CD', 'Not-CD'], array([[2, 3],
       [1, 6]]), 0.47916666666666663, RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), {'mean_test_f1_micro': array([0.58333333, 0.5       , 0.41666667, 0.41666667]), 'split0_test_accuracy': array([0.5, 0.5, 0.5, 0.5]), 'mean_test_precision_micro': array([0.58333333, 0.5       , 0.41666667, 0.41666667]), 'split1_test_roc_auc': array([0.5  , 0.75 , 0.5  , 0.625]), 'split0_test_precision_macro': array([0.25, 0.25, 0.25, 0.25]), 'rank_test_roc_auc': array([4, 2, 

In [4]:
################ Support Vector Machine ################
MSVM = SVM(X, Y)
MSVM.tune_and_eval('../../crohns_disease/results-PRUEBA/SVMclassifier', n_fold=2)

/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[['CD', 'Not-CD'], array([[0, 5],
       [1, 6]]), 0.5809523809523809, LinearSVC(C=200, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0), {'param_C': masked_array(data=[1000, 500, 200, 100, 50, 20, 10, 5, 2, 1, 0.2, 0.5,
                   0.01, 0.02, 0.05, 0.001],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'mean_test_f1_micro': array([0.41666667, 0.58333333, 0.66666667, 0.41666667, 0.41666667,
       0.41666667, 0.41666667, 0.5       , 0.41666667, 0.5       ,
       0.41666667, 0.41666667, 0.41666667, 0.41666667, 0.41666667,
       0.41666667]), 'split0_test_accuracy': array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5]), 'mean

#### DNN Classifier

In [2]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')

In [3]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Multilayer Perceptron  ################
DNN=DNNMutliclass16S(X, Y, model_strct='mlp', model_arch=[1024,0.2,512,0.2,256,0.1,128,8])
#DNN=DNNMutliclass16S(X,Y,model_strct='mlp',model_arch=[10,0.2,10,4])
#DNN.cross_validation('../../crohns_disease/results-PRUEBA/DNNclassifier/', gpu_dev='1', n_fold=10, epochs=100, batch_size=1000)
DNN.cross_validation('../../crohns_disease/results-PRUEBA/DNNclassifier/', gpu_dev='1', n_fold=2, epochs=1, batch_size=6)
#X_new=DNNMutliclass16S.make_activation_function('../../crohns_disease/results-PRUEBA/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.37.pickle',X)


 Evaluation on a new fold is now get started ..
6/6 [==============================] - 0s

 Evaluation on a new fold is now get started ..


/home/nuria/.local/share/virtualenvs/MicroPheno-f1MtFhBm/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s
['0.42 $\\pm$ 0.08 & 0.42 $\\pm$ 0.08 & 0.42 $\\pm$ 0.08 & 0.21 $\\pm$ 0.04 & 0.5 $\\pm$ 0.0 & 0.29 $\\pm$ 0.04 & 0.42 $\\pm$ 0.08 & 0.5 $\\pm$ 0.0', [0.5, 0.3333333333333333], [0.5, 0.3333333333333333], [0.5, 0.3333333333333333], [0.25, 0.16666666666666666], [0.5, 0.5], [0.3333333333333333, 0.25], [0.5, 0.3333333333333333], [0.5, 0.5], ([0.6931056976318359], [0.6941101551055908], range(1, 2))]


/home/nuria/.local/share/virtualenvs/MicroPheno-f1MtFhBm/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


12/12 [==============================] - 0s
[[array([[-0.02568717, -0.00824866, -0.02018957, ...,  0.00865864,
         0.0320547 , -0.00824117],
       [ 0.02163045, -0.00933056,  0.01645522, ...,  0.01403824,
         0.00529485, -0.0134734 ],
       [ 0.02662799,  0.01257598, -0.00159549, ...,  0.03011716,
         0.00408212, -0.01451909],
       ...,
       [ 0.00915833,  0.03240604, -0.03203336, ...,  0.00909885,
         0.00879921, -0.0055354 ],
       [-0.01453778, -0.01458047, -0.02400308, ...,  0.02097211,
         0.00366616,  0.01401315],
       [ 0.02640337, -0.02194849,  0.01755379, ...,  0.0322344 ,
        -0.02254829,  0.02135   ]], dtype=float32), array([0.0000000e+00, 0.0000000e+00, 2.1285785e-05, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)], [], [array([[-4.21432704e-02,  2.74246186e-02, -1.00354105e-02, ...,
        -4.08504158e-02,  1.22201890e-02,  2.37293392e-02],
       [-5.81039935e-02, -1.21877491e-02,  5.84055334e-02, ...,
     

### Mostrar as métricas obtidas

In [8]:
### RF
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/filtered_metrics.csv")
metrics

,Unnamed: 0,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,mean_test_roc_auc,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.479167,0.583333,0.4375,0.583333,0.5625,0.583333,0.555556,0.083333,0.145833,0.083333,0.1875,0.083333,0.0625,0.083333,0.055556
1,1,0.409524,0.500000,0.3750,0.500000,0.5000,0.500000,0.708333,0.000000,0.076190,0.000000,0.1250,0.000000,0.0000,0.000000,0.041667
2,2,0.333333,0.416667,0.3125,0.416667,0.4375,0.416667,0.611111,0.083333,0.000000,0.083333,0.0625,0.083333,0.0625,0.083333,0.111111
3,3,0.333333,0.416667,0.3125,0.416667,0.4375,0.416667,0.756944,0.083333,0.000000,0.083333,0.0625,0.083333,0.0625,0.083333,0.131944


In [9]:
### SVM
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/SVMclassifier/filtered_metrics.csv")
metrics

,Unnamed: 0,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,mean_test_roc_auc,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
1,1,0.366667,0.583333,0.291667,0.583333,0.5000,0.583333,0.770833,0.083333,0.033333,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
2,2,0.580952,0.666667,0.541667,0.666667,0.6875,0.666667,0.770833,0.166667,0.247619,0.166667,0.291667,0.166667,0.1875,0.166667,0.104167
3,3,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
4,4,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
5,5,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
6,6,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
7,7,0.409524,0.500000,0.475000,0.500000,0.5625,0.500000,0.770833,0.000000,0.076190,0.000000,0.225000,0.000000,0.0625,0.000000,0.104167
8,8,0.291667,0.416667,0.208333,0.416667,0.5000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.0000,0.083333,0.104167
9,9,0.409524,0.500000,0.475000,0.500000,0.5625,0.500000,0.770833,0.000000,0.076190,0.000000,0.225000,0.000000,0.0625,0.000000,0.104167


In [10]:
### MLP
#[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, 
# (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-ARTICULO/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, accuracy, roc_auc,
 (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-PRUEBA/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.37.pickle')

#p_micro
#r_micro
#f1_micro
#p_macro
#r_macro
#f1_macro
accuracy
#roc_auc

[0.5, 0.6666666666666666]

# Probas varias

In [ ]:
# proba MLP
#z=FileUtility.load_obj('../../crohns_disease/results/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')
#t=FileUtility.load_sparse_csr('../../crohns_disease/results/DNNclassifier/activationlayer_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

#print(z)

###########################################################################################

#[label_set, conf, best_score_, best_estimator_, cv_results_, best_params_, predictions]=FileUtility.load_obj('../../crohns_disease/results-PRUEBA/RFclassifier/all_results.pickle')

#results=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/cv_results_.csv")

#results.columns

#results[['mean_test_f1_macro', 'mean_test_f1_micro', 'mean_test_precision_macro', 
#       'mean_test_precision_micro', 'mean_test_recall_macro', 'mean_test_recall_micro',
#       'std_test_f1_macro', 'std_test_f1_micro', 'std_test_precision_macro',
#       'std_test_precision_micro', 'std_test_recall_macro', 'std_test_recall_micro']]


#results.drop(['mean_fit_time', 'mean_score_time', 'std_fit_time', 'std_score_time', 
#              'param_max_features', 'param_min_samples_leaf', 'param_min_samples_split',
#              'param_criterion', 'param_n_estimators', 'rank_test_f1_macro', 'rank_test_f1_micro',
#              'rank_test_precision_macro', 'rank_test_precision_micro', 'rank_test_recall_macro',
#              'rank_test_recall_micro'], axis=1)

###########################################################################################

#[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, 
# (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-ARTICULO/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

###########################################################################################3

#print(sys.version)
#import pkg_resources
#installed_packages = pkg_resources.working_set
#installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
#for i in installed_packages])
#print(installed_packages_list)